In [1]:
import sys
import os
import math
import numpy as np
import galsim as galsim
import galsim.wfirst as wfirst

In [2]:
use_filters = None

In [3]:
random_seed = 1324789
rng = galsim.BaseDeviate(random_seed)

In [4]:
poisson_noise = galsim.PoissonNoise(rng)

In [5]:
filters = wfirst.getBandpasses(AB_zeropoint=True)

In [6]:
use_SCA = 7
PSFs = wfirst.getPSF(SCAs=use_SCA, approximate_struts=True, n_waves=10)
PSF = PSFs[use_SCA]

In [27]:
stamp_star = 128
stamp_size = 1536
stamp_star_big = 512

In [22]:
ra_targ = 30.*galsim.degrees
dec_targ = -60.*galsim.degrees
targ_pos = galsim.CelestialCoord(ra=ra_targ, dec=dec_targ)
wcs_list = wfirst.getWCS(world_pos=targ_pos, SCAs=use_SCA)
wcs = wcs_list[use_SCA]
SCA_cent_pos = wcs.toWorld(galsim.PositionD(wfirst.n_pix/2, wfirst.n_pix/2))

In [23]:
%run M65.ipynb
%run M66.ipynb
%run NGC3628.ipynb

In [32]:
n = 50
shift_star = np.random.rand(n, 2) * 3800 - np.ones((n, 2)) * 1900

In [25]:
hst_eff_area = 2.4**2 * (1.-0.33**2)
wfirst_eff_area = galsim.wfirst.diameter**2 * (1.-galsim.wfirst.obscuration**2)
flux_scaling = (wfirst_eff_area/hst_eff_area) * wfirst.exptime
print(flux_scaling)

137.76328140781058


In [33]:
for filter_name, filter_ in filters.items():
    if use_filters is not None and filter_name[0] not in use_filters:
        continue
    file_name = 'demo13_PSF_{0}.fits'.format(filter_name)
    point = galsim.DeltaFunction(flux=1.)
    star_sed = galsim.SED(lambda x:1, 'nm', 'flambda').withFlux(1.,filter_)
    star = galsim.Convolve(point*star_sed, PSF)
    img_psf = galsim.ImageF(64, 64)
    star.drawImage(bandpass=filter_, image=img_psf, scale=wfirst.pixel_scale)
    img_psf.write(file_name)
    
    final_image = galsim.ImageF(wfirst.n_pix, wfirst.n_pix, wcs=wcs)
    
    star_sed = galsim.SED(lambda x:1, 'nm', 'flambda').withFlux(1.e4,filter_)
    for a in range(n):
        shift = shift_star[a,:]
        x = np.round(shift[0])
        y = np.round(shift[1])
        star_final = galsim.Convolve(flux_scaling*point*star_sed, PSF)
        b = galsim.BoundsI(xmin=2048+x-(stamp_star/2)+1,
                          xmax=2048+x+(stamp_star/2),
                          ymin=2048+y-(stamp_star/2)+1,
                          ymax=2048+y+(stamp_star/2))
        star_final.drawImage(bandpass=filter_, image=final_image[b], scale=wfirst.pixel_scale, add_to_image=True)
    
    gal_sed = galsim.SED(lambda x:1, 'nm', 'flambda').withFlux(7.e5,filter_)
    star_sed = galsim.SED(lambda x:1, 'nm', 'flambda').withFlux(1.e5,filter_)
    b_M65 = galsim.BoundsI(xmin=2848-(stamp_size/2)+1,
                          xmax=2848+(stamp_size/2),
                          ymin=2848-(stamp_size/2)+1,
                          ymax=2848+(stamp_size/2))
    M65_final = galsim.Convolve(flux_scaling*M65*gal_sed, PSF)
    M65_final.drawImage(bandpass=filter_, image=final_image[b_M65], scale=wfirst.pixel_scale, add_to_image=True)
    b_M66 = galsim.BoundsI(xmin=3148-(stamp_size/2)+1,
                          xmax=3148+(stamp_size/2),
                          ymin=1548-(stamp_size/2)+1,
                          ymax=1548+(stamp_size/2))
    M66_final = galsim.Convolve(flux_scaling*M66*gal_sed, PSF)
    M66_final.drawImage(bandpass=filter_, image=final_image[b_M66], scale=wfirst.pixel_scale, add_to_image=True)
    b_NGC3628 = galsim.BoundsI(xmin=1048-(stamp_size/2)+1,
                              xmax=1048+(stamp_size/2),
                              ymin=1548-(stamp_size/2)+1,
                              ymax=1548+(stamp_size/2))
    NGC3628_final = galsim.Convolve(flux_scaling*NGC3628*gal_sed, PSF)
    NGC3628_final.drawImage(bandpass=filter_, image=final_image[b_NGC3628], scale=wfirst.pixel_scale, add_to_image=True)
    b_star = galsim.BoundsI(xmin=1748-(stamp_star_big/2)+1,
                           xmax=1748+(stamp_star_big/2),
                           ymin=2748-(stamp_star_big/2)+1,
                           ymax=2748+(stamp_star_big/2))
    star_final = galsim.Convolve(flux_scaling*point*star_sed, PSF)
    star_final.drawImage(bandpass=filter_, image=final_image[b_star], scale=wfirst.pixel_scale, add_to_image=True)
    
    file_name = 'demo13_LeoTrio.fits'
    final_image.write(file_name)
    
    sky_level = wfirst.getSkyLevel(filters[filter_name], world_pos=SCA_cent_pos)
    sky_level *= (1.0 + wfirst.stray_light_fraction)
    sky_image = final_image.copy()
    wcs.makeSkyImage(sky_image, sky_level)
    sky_image += wfirst.thermal_backgrounds[filter_name]*wfirst.exptime
    final_image += sky_image
    
    final_image.addNoise(poisson_noise)
    
    save_image = final_image.copy()
    
    wfirst.addReciprocityFailure(final_image)
    
    diff = final_image-save_image
    out_filename = 'demo13_RecipFail_{0}.fits'.format(filter_name)
    final_image.write(out_filename)
    out_filename = 'demo13_diff_RecipFail_{0}.fits'.format(filter_name)
    diff.write(out_filename)
    
    final_image.quantize()
    
    dark_current = wfirst.dark_current*wfirst.exptime
    dark_noise = galsim.DeviateNoise(galsim.PoissonDeviate(rng, dark_current))
    final_image.addNoise(dark_noise)
    
    save_image = final_image.copy()
    
    wfirst.applyNonlinearity(final_image)
    
    diff = final_image-save_image
    out_filename = 'demo13_NL_{0}.fits'.format(filter_name)
    final_image.write(out_filename)
    out_filename = 'demo13_diff_NL_{0}.fits'.format(filter_name)
    diff.write(out_filename)
    save_image = final_image.copy()
    
    wfirst.applyIPC(final_image)
    
    diff = final_image-save_image
    out_filename = 'demo13_IPC_{0}.fits'.format(filter_name)
    final_image.write(out_filename)
    out_filename = 'demo13_diff_IPC_{0}.fits'.format(filter_name)
    diff.write(out_filename)
    
    read_noise = galsim.GaussianNoise(rng, sigma=wfirst.read_noise)
    final_image.addNoise(read_noise)
    
    final_image /= wfirst.gain
    
    final_image.quantize()
    
    sky_image.quantize()
    tot_sky_image = (sky_image + round(dark_current))/wfirst.gain
    tot_sky_image.quantize()
    final_image -= tot_sky_image
    
    out_filename = 'demo13_{0}.fits'.format(filter_name)
    final_image.write(out_filename)